In [89]:
from __future__ import division, print_function, unicode_literals

In [90]:
%tensorflow_version 1.x
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pickle
import csv
import numpy as np
import tensorflow as tf
import pandas as pd
import httplib2
import os
import requests
from PIL import Image
import time
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [91]:
from google.colab import drive
drive.mount("/content/gdrive/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [92]:
tf.reset_default_graph()

In [93]:
np.random.seed(42)
tf.set_random_seed(42)

# Load Data


In [94]:
# !unzip 'gdrive/My Drive/Covid19/images/pCT_preprocess.zip' -d 'gdrive/My Drive/Covid19/images/pCT_preprocess'
# !unzip 'gdrive/My Drive/Covid19/images/nCT2_preprocess.zip' -d 'gdrive/My Drive/Covid19/images/nCT_preprocess'
# !unzip 'gdrive/My Drive/Covid19/images/NiCT_preprocess.zip' -d 'gdrive/My Drive/Covid19/images/NiCT_preprocess'

# # count of data on each folder
# !ls  "gdrive/My Drive/Covid19/images/NiCT_preprocess/NiCT_preprocess" | wc -l

In [95]:
checkpoint_path = 'gdrive/My Drive/Covid19/codes/CAPS/Prot2_IR5/'

image_size = 112
# image_size = 28
img_channel = 1
img_no = 19685
num_pct = 4001
num_nct = 9979
num_nict = 5705

pathMain = 'gdrive/My Drive/Covid19/images/'

from tqdm.notebook import tqdm
if False:
  All = np.zeros( shape=(img_no,image_size,image_size) )

  cnt = 0
  for i in tqdm(range(num_pct)):
    path = pathMain + 'pCT_preprocess/pCT_preprocess/pCT'
    path = path +  str(i+1) + '.jpg'
    I = Image.open(path)
    I.thumbnail((image_size, image_size), Image.ANTIALIAS)
    I = np.reshape(I,(1,image_size,image_size))
    All[cnt] = I
    cnt += 1
  for i in tqdm(range(num_nct)):
    path = pathMain + 'nCT_preprocess/nCT2_preprocess/nCT'
    path = path +  str(i+1) + '.jpg'
    I = Image.open(path)
    I.thumbnail((image_size, image_size), Image.ANTIALIAS)
    I = np.reshape(I,(1,image_size,image_size))
    All[cnt] = I
    cnt += 1
  for i in tqdm(range(num_nict)):
    path = pathMain + 'NiCT_preprocess/NiCT_preprocess/NiCT'
    path = path +  str(i+1) + '.jpg'
    I = Image.open(path)
    I.thumbnail((image_size, image_size), Image.ANTIALIAS)
    I = np.reshape(I,(1,image_size,image_size))
    All[cnt] = I
    cnt += 1

  with open('gdrive/My Drive/Covid19/targets_prot2.pckl', 'rb') as f:
    All_label = pickle.load(f)

  All_label = np.argmax(All_label , axis = 1)
  

  # with open(pathMain + 'DS.pckl', 'wb') as f:
  #   pickle.dump([All, All_label], f)

else:
  with open(pathMain + 'DS.pckl', 'rb') as f:
      All, All_label = pickle.load(f)

from sklearn.model_selection import train_test_split
Train, Test, Train_label, Test_label = train_test_split(All, All_label, test_size=0.25, random_state=101, stratify=All_label)

pct_no = 2000
Train_pct = Train[Train_label==0]  
Train = Train[Train_label != 0]
y_train_pct = Train_label[Train_label==0]
Train_label = Train_label[Train_label != 0]
Train_pct = Train_pct[:pct_no]
y_train_pct = y_train_pct[:pct_no]

Train = np.concatenate((Train,Train_pct),axis=0)
Train_label = np.concatenate((Train_label,y_train_pct),axis=0)

In [96]:
Train = Train / 255.0
Test = Test / 255.0
Train.mean()

0.6790533871547073

In [97]:
num_class = len(set(Train_label))

In [98]:
X = tf.placeholder(shape=[None, image_size, image_size, img_channel], dtype=tf.float32, name="X")

# Primary Capsules

The first layer will be composed of 32 maps of 6×6 capsules each, where each capsule will output an 8D activation vector:

In [99]:
caps1_n_maps = 32
# caps1_n_caps = caps1_n_maps * 6 * 6  # 1152 primary capsules  (FOR MNIST)
caps1_n_caps = caps1_n_maps * 8 * 8  # 41472 primary capsules    (FOR Cedar)
# caps1_n_caps = caps1_n_maps * 12 * 12  # 41472 primary capsules    (FOR Cedar)

caps1_n_dims = 8

In [100]:
conv1_params = {
    "filters": 256,
    "kernel_size": 17,
    "strides": 2,
    "padding": "valid",
    "activation": tf.nn.relu,
}

conv2_params = {
    "filters": caps1_n_maps * caps1_n_dims, # 256 convolutional filters
    "kernel_size": 17,
    "strides": 4,
    "padding": "valid",
    "activation": tf.nn.relu
}

In [101]:
conv1 = tf.layers.conv2d(X, name="conv1", **conv1_params)
conv2 = tf.layers.conv2d(conv1, name="conv2", **conv2_params)

In [102]:
caps1_raw = tf.reshape(conv2, [-1, caps1_n_caps, caps1_n_dims],
                       name="caps1_raw")

In [103]:
def squash(s, axis=-1, epsilon=1e-7, name=None):
    with tf.name_scope(name, default_name="squash"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keep_dims=True)
        safe_norm = tf.sqrt(squared_norm + epsilon)
        squash_factor = squared_norm / (1. + squared_norm)
        unit_vector = s / safe_norm
        return squash_factor * unit_vector

In [104]:
caps1_output = squash(caps1_raw, name="caps1_output")

# Digit Capsules

## Compute the Predicted Output Vectors

In [105]:
caps2_n_caps = num_class
caps2_n_dims = 16

In [106]:
init_sigma = 0.1

W_init = tf.random_normal(
    shape=(1, caps1_n_caps, caps2_n_caps, caps2_n_dims, caps1_n_dims),
    stddev=init_sigma, dtype=tf.float32, name="W_init")
W = tf.Variable(W_init, name="W")

In [107]:
batch_size = tf.shape(X)[0]
W_tiled = tf.tile(W, [batch_size, 1, 1, 1, 1], name="W_tiled")

In [108]:
caps1_output_expanded = tf.expand_dims(caps1_output, -1,
                                       name="caps1_output_expanded")
caps1_output_tile = tf.expand_dims(caps1_output_expanded, 2,
                                   name="caps1_output_tile")
caps1_output_tiled = tf.tile(caps1_output_tile, [1, 1, caps2_n_caps, 1, 1],
                             name="caps1_output_tiled")

In [109]:
caps2_predicted = tf.matmul(W_tiled, caps1_output_tiled,
                            name="caps2_predicted")

## Routing by agreement

In [110]:
raw_weights = tf.zeros([batch_size, caps1_n_caps, caps2_n_caps, 1, 1],
                       dtype=np.float32, name="raw_weights")

### Round 1

In [111]:
routing_weights = tf.nn.softmax(raw_weights, dim=2, name="routing_weights")

In [112]:
weighted_predictions = tf.multiply(routing_weights, caps2_predicted,
                                   name="weighted_predictions")
weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, keep_dims=True,
                             name="weighted_sum")

In [113]:
caps2_output_round_1 = squash(weighted_sum, axis=-2,
                              name="caps2_output_round_1")

### Round 2

In [114]:
caps2_output_round_1_tiled = tf.tile(
    caps2_output_round_1, [1, caps1_n_caps, 1, 1, 1],
    name="caps2_output_round_1_tiled")

In [115]:
agreement = tf.matmul(caps2_predicted, caps2_output_round_1_tiled,
                      transpose_a=True, name="agreement")

In [116]:
raw_weights_round_2 = tf.add(raw_weights, agreement,
                             name="raw_weights_round_2")

The rest of round 2 is the same as in round 1:

In [117]:
routing_weights_round_2 = tf.nn.softmax(raw_weights_round_2,
                                        dim=2,
                                        name="routing_weights_round_2")
weighted_predictions_round_2 = tf.multiply(routing_weights_round_2,
                                           caps2_predicted,
                                           name="weighted_predictions_round_2")
weighted_sum_round_2 = tf.reduce_sum(weighted_predictions_round_2,
                                     axis=1, keep_dims=True,
                                     name="weighted_sum_round_2")
caps2_output_round_2 = squash(weighted_sum_round_2,
                              axis=-2,
                              name="caps2_output_round_2")

In [118]:
caps2_output = caps2_output_round_2

# Estimated Class Probabilities (Length)

In [119]:
def safe_norm(s, axis=-1, epsilon=1e-7, keep_dims=False, name=None):
    with tf.name_scope(name, default_name="safe_norm"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keep_dims=keep_dims)
        return tf.sqrt(squared_norm + epsilon)

In [120]:
y_proba = safe_norm(caps2_output, axis=-2, name="y_proba")

In [121]:
y_proba_argmax = tf.argmax(y_proba, axis=2, name="y_proba")

In [122]:
y_pred = tf.squeeze(y_proba_argmax, axis=[1,2], name="y_pred")

# Labels

In [123]:
y = tf.placeholder(shape=[None], dtype=tf.int64, name="y")

# Margin loss

In [124]:
m_plus = 0.9
m_minus = 0.1
lambda_ = 0.02

In [125]:
T = tf.one_hot(y, depth=caps2_n_caps, name="T")

In [126]:
caps2_output_norm = safe_norm(caps2_output, axis=-2, keep_dims=True,
                              name="caps2_output_norm")

In [127]:
present_error_raw = tf.square(tf.maximum(0., m_plus - caps2_output_norm),
                              name="present_error_raw")
present_error = tf.reshape(present_error_raw, shape=(-1, num_class),
                           name="present_error")

In [128]:
absent_error_raw = tf.square(tf.maximum(0., caps2_output_norm - m_minus),
                             name="absent_error_raw")
absent_error = tf.reshape(absent_error_raw, shape=(-1, num_class),
                          name="absent_error")

In [129]:
L = tf.add(T * present_error, lambda_ * (1.0 - T) * absent_error,
           name="L")

In [130]:
C = np.zeros((1 , num_class),dtype=np.float32)
for i in range(num_class):
  C[0,i] = 1 - np.sum(Train_label == i) / len(Train_label)

L_CS = tf.multiply(L , tf.constant(C))

# margin_loss = tf.reduce_mean(tf.reduce_sum(L, axis=1), name="margin_loss")
margin_loss = tf.reduce_mean(tf.reduce_sum(L_CS, axis=1), name="margin_loss")

## Final Loss

In [131]:
alpha = 0.00001

regularizer = tf.nn.l2_loss(W_tiled)
beta = 0#0.000001
##loss = tf.add(loss_, beta * regularizer, name="loss")
loss = tf.add(margin_loss, beta * regularizer, name="loss")

# loss_ = tf.add(margin_loss, alpha * reconstruction_loss, name="loss_")
# regularizer = tf.nn.l2_loss(W_tiled)
# beta = 0.0000001
# loss = tf.add(loss_, beta * regularizer, name="loss")

loss_for_plot = tf.add(margin_loss, beta * regularizer / tf.cast(batch_size, tf.float32), name="loss") #**

## Accuracy

In [132]:
correct = tf.equal(y, y_pred, name="correct")
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

## Training Operations

In [133]:
optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss, name="training_op")


In [134]:
init = tf.global_variables_initializer()
saver = tf.train.Saver( max_to_keep =1 , filename = 'TestName')

In [135]:
def getNextBatchTrain(batch_size):
  N = np.size(Train,0)
  idx = np.random.randint(0,N,batch_size)
  batchLabel = Train_label[idx]
  return Train[idx,:]  , batchLabel.astype('uint8')

In [136]:
def getNextBatchTest(batch_size):
  N = np.size(Test,0)
  idx = np.random.randint(0,N,batch_size)
  batchLabel = Test_label[idx]
  return Test[idx,:]  , batchLabel.astype('uint8')

# Training

In [137]:
# n_epochs = 100
# batch_size = 10
# restore_checkpoint = False

# n_iterations_per_epoch = len(Train_label) // batch_size
# n_iterations_validation = len(Test_label)
# best_loss_val = np.infty
# checkpoint_path = "/content/gdrive/My Drive/Dataset/covid19/"

# loss_trains = []
# with tf.Session() as sess:
#     if restore_checkpoint and tf.train.checkpoint_exists(checkpoint_path):
#         saver.restore(sess, checkpoint_path)
#     else:
#         print('not loaded!')
#         init.run()

#     for epoch in range(n_epochs):
#       for iteration in range(1, n_iterations_per_epoch + 1):
#           X_batch, y_batch = getNextBatchTrain(batch_size)
#           # Run the training operation and measure the loss:
#           _, loss_train = sess.run(
#               [training_op, loss],
#               feed_dict={X: X_batch.reshape([-1, image_size, image_size, img_channel]),
#                           y: y_batch})
#           print("\rIteration: {}/{} ({:.1f}%)  Loss: {:.5f}".format(
#                     iteration, n_iterations_per_epoch,
#                     iteration * 100 / n_iterations_per_epoch,
#                     loss_train),
#                 end="")

#       loss_trains.append(loss_train)
#       # At the end of each epoch,
#       # measure the validation loss and accuracy:
#       loss_vals = []
#       acc_vals = []
#       y_pred_all = []
#       for iteration in range(1, n_iterations_validation + 1):
#         X_batch = Test[iteration-1:iteration] #getNextBatchTest(batch_size)
#         y_batch = Test_label[iteration-1:iteration].astype('uint8')
#         loss_val, acc_val , y_pred_sample = sess.run(
#                 [loss, accuracy, y_pred],
#                 feed_dict={X: X_batch.reshape([-1, image_size, image_size, img_channel]),
#                           y: y_batch})
#         loss_vals.append(loss_val)
#         acc_vals.append(acc_val)
#         y_pred_all.append(y_pred_sample)
      
#       print("\rEvaluating the model: {}/{} ({:.1f}%)".format(
#                 iteration, n_iterations_validation,
#                 iteration * 100 / n_iterations_validation),
#             end=" " * 10)
#       loss_val = np.mean(loss_vals)
#       acc_val = np.mean(acc_vals)
#       print("\rEpoch: {}  Val accuracy: {:.4f}%  Loss: {:.6f}{}".format(
#           epoch + 1, acc_val * 100, loss_val,
#           " (improved)" if loss_val < best_loss_val else ""))

#       print(loss_train)
#       # And save the model if it improved: (No!)
#       if False:
#         save_path = saver.save(sess, checkpoint_path)
#       # if loss_val < best_loss_val:
#       #   # clearCheckPointFiles()
#       #   save_path = saver.save(sess, checkpoint_path);
#       #   best_loss_val = loss_val

In [ ]:

n_epochs = 100
batch_size = 10
restore_checkpoint = True

n_iterations_per_epoch = len(Train_label) // batch_size
n_iterations_validation = len(Test_label) #// batch_size
best_loss_val = np.infty

with tf.Session() as sess:
    if restore_checkpoint and tf.train.checkpoint_exists(checkpoint_path):
        saver.restore(sess, checkpoint_path)
        with open(checkpoint_path + 'OtherVars', 'rb') as f:
            acc_trains, loss_trains, acc_tests, loss_tests ,time_per_epochs, start_epoch , y_pred_all = pickle.load(f)
            start_epoch += 1
        print('\nStarting from epoch: %.0f\n' %(start_epoch + 1))
    else:
        print('\nCheck point not loaded\n')
        init.run()
        loss_trains = []
        acc_trains = [] 
        loss_tests = []
        acc_tests = []
        time_per_epochs = []
        start_epoch = 0
        y_pred_all = np.zeros((n_iterations_validation,n_epochs))

    for epoch in range(n_epochs):
        startTime = time.time()
        loss_train = []
        acc_train=[]
        for iteration in range(1, n_iterations_per_epoch + 1):
            X_batch, y_batch = getNextBatchTrain(batch_size)
            # Run the training operation and measure the loss:
            _, loss_train_batch,acc_train_batch = sess.run(
                [training_op, loss_for_plot,accuracy],
                feed_dict = {X: X_batch.reshape([-1, image_size, image_size, img_channel]),
                           y: y_batch})
            print("\rIteration: {}/{} ({:.1f}%)  Loss: {:.5f}".format(
                      iteration, n_iterations_per_epoch,
                      iteration * 100 / n_iterations_per_epoch,
                      loss_train_batch),
                  end="")
            
            loss_train.append(loss_train_batch)
            acc_train.append(acc_train_batch)

        end_time = time.time()
        time_per_epochs.append(end_time - startTime)
        print('\nElapsed: %.1f' % (end_time - startTime))
        remainHour = (n_epochs-epoch) * (end_time - startTime)/3600
        print('Estimated remaining time: %.1f hours' % remainHour)

        acc_trains.append(np.mean(acc_train))
        loss_trains.append(np.mean(loss_train))

        #print("*****")
        #print("loss_train:",np.mean(loss_train)) #**        
        #print("acc_train",np.mean(acc_train)*100) #**Javidi
        #print("*****")

        # At the end of each epoch,
        # measure the validation loss and accuracy:
        loss_vals = []
        acc_vals = []
        
        for iteration in range(1, n_iterations_validation + 1):
            X_batch = Test[iteration-1:iteration]
            y_batch = Test_label[iteration-1:iteration].astype('uint8')
            #X_batch, y_batch = getNextBatchTest(batch_size)
            loss_val, acc_val , y_pred_sample = sess.run(
                    [loss_for_plot, accuracy , y_pred],
                    feed_dict={X: X_batch.reshape([-1, image_size, image_size, img_channel]),
                               y: y_batch})

            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
            y_pred_all[iteration-1 , epoch] =  y_pred_sample

            print("\rEvaluating the model: {}/{} ({:.1f}%)".format(
                      iteration, n_iterations_validation,
                      iteration * 100 / n_iterations_validation),
                  end=" " * 10)
            
        loss_val = np.mean(loss_vals)
        acc_val = np.mean(acc_vals)

        print("\rEpoch: {}  Train accuracy: {:.4f}%  Loss_train: {:.6f}  Val accuracy: {:.4f}%  Loss_test: {:.6f}{}".format(
            epoch + 1, np.mean(acc_train)*100,np.mean(loss_train),acc_val * 100, loss_val,
            " (improved)" if loss_val < best_loss_val else ""))

        loss_tests.append(loss_val) #**
        acc_tests.append(acc_val) #**

        #print(np.mean(loss_trains)) #**        
        

        #**
        np.savetxt(checkpoint_path+"loss_tr.csv", loss_trains, delimiter=",")
        np.savetxt(checkpoint_path+"loss_te.csv", loss_tests, delimiter=",")
        np.savetxt(checkpoint_path+"acc_tr.csv", acc_trains, delimiter=",")
        np.savetxt(checkpoint_path+"acc_te.csv", acc_tests, delimiter=",")

        # Save model all the time
        save_path = saver.save(sess, checkpoint_path)

        with open(checkpoint_path + 'OtherVars', 'wb') as f:
            start_epoch = epoch
            pickle.dump([acc_trains, loss_trains, acc_tests, loss_tests ,time_per_epochs, start_epoch,y_pred_all], f)
        


Check point not loaded

Iteration: 1376/1376 (100.0%)  Loss: 0.02256
Elapsed: 210.6
Estimated remaining time: 5.8 hours
Epoch: 1  Train accuracy: 42.5073%  Loss_train: 0.023110  Val accuracy: 50.6908%  Loss_test: 0.022573 (improved)
Iteration: 1376/1376 (100.0%)  Loss: 0.02231
Elapsed: 182.0
Estimated remaining time: 5.0 hours
Epoch: 2  Train accuracy: 31.4971%  Loss_train: 0.022881  Val accuracy: 28.9923%  Loss_test: 0.022595 (improved)
Iteration: 1376/1376 (100.0%)  Loss: 0.02182
Elapsed: 183.4
Estimated remaining time: 5.0 hours
Epoch: 3  Train accuracy: 31.1991%  Loss_train: 0.022874  Val accuracy: 28.9923%  Loss_test: 0.022593 (improved)
Iteration: 1376/1376 (100.0%)  Loss: 0.02257
Elapsed: 182.6
Estimated remaining time: 4.9 hours
Epoch: 4  Train accuracy: 31.4462%  Loss_train: 0.022837  Val accuracy: 28.9923%  Loss_test: 0.022590 (improved)
Iteration: 1376/1376 (100.0%)  Loss: 0.02386
Elapsed: 181.9
Estimated remaining time: 4.9 hours
Epoch: 5  Train accuracy: 38.7573%  Loss_tr

In [ ]:
print(confusion_matrix(Test_label, y_pred_all[:,epoch]))
print(classification_report(Test_label, y_pred_all[:,epoch]))
